In [ ]:
# def cohesion_index_intra(buildings_gdf):
#     # drop empty / zero-area pieces
#     buildings_gdf = buildings_gdf[~buildings_gdf.geometry.is_empty]
#     buildings_gdf = buildings_gdf[buildings_gdf.geometry.area > 0]
#     n = len(buildings_gdf)
#     if n < 2:
#         return 0 if n == 1 else None
# 
#     areas = buildings_gdf.geometry.area.to_numpy()
#     centroids = np.array([[pt.x, pt.y] for pt in buildings_gdf.geometry.centroid])
#     total_area = areas.sum()
# 
#     # pairwise distances and area interaction
#     D = distance_matrix(centroids, centroids)          # n x n
#     A_outer = np.outer(areas, areas)                   # n x n
#     interaction_sum = np.sum(np.triu(A_outer * D, 1))  # sum only upper triangle (unique pairs)
# 
#     denominator = total_area * n * (n - 1)
#     return (2 * interaction_sum) / denominator
# 
# if "ci_intra" not in ABT.columns:
#     
#     # --- Step 2: Results container ---
#     results = []
#     
#     # --- Step 3: Loop through subdivisions ---
#     for idx, sub in tqdm(ABT.iterrows(), total=len(ABT), desc="Calculating CI_intra"):
#         sub_id = sub["subd_id"]
#         sub_geom = sub.geometry
#     
#         # buildings that intersect the subdivision (fully or partially)
#         intersecting_buildings = building_footprint_dataset[building_footprint_dataset.intersects(sub_geom)]
#     
#         if len(intersecting_buildings) == 0:
#             ci_value = None
#         else:
#             # clip each building to the subdivision boundary
#             clipped = intersecting_buildings.copy()
#             clipped["geometry"] = clipped.geometry.intersection(sub_geom)
#     
#             # remove empty geometries created by intersection
#             clipped = clipped[~clipped.geometry.is_empty & clipped.geometry.is_valid]
#     
#             ci_value = cohesion_index_intra(clipped) if len(clipped) > 0 else None
#     
#         results.append({"subd_id": sub_id, "ci_intra": ci_value})
#     
#     # --- Step 4: Merge results back ---
#     ci_df = gpd.GeoDataFrame(results)
#     ABT = ABT.merge(ci_df, on="subd_id", how="left")
#     ABT["ci_intra"] = ABT["ci_intra"].astype(float).round(2)
#     
#     print("✅ Cohesion Index (Intra) computed for all subdivisions (partial buildings included).")
# 
# else:
#     print("✅ Cohesion Index (Intra) already exist).")

In [ ]:
if "ci_intra" not in ABT.columns:
    
    # # -----------------------------
    # # Step 1: Rasterize building footprints
    # # -----------------------------
    # def rasterize_buildings(buildings, subdivision, resolution=5):
    #     """
    #     Rasterizes building footprints within a subdivision.
    # 
    #     Parameters:
    #         buildings (list of shapely Polygons): Building footprints
    #         subdivision (shapely Polygon): Subdivision boundary
    #         resolution (float): raster cell size (same units as CRS)
    # 
    #     Returns:
    #         raster (np.array): 2D array where 1 = building cell, 0 = empty
    #         transform (affine): rasterio transform for coordinates
    #     """
    #     if len(buildings) == 0:
    #         return np.zeros((1, 1), dtype=np.uint8), None
    # 
    #     buildings_union = unary_union(buildings)
    #     minx, miny, maxx, maxy = subdivision.bounds
    #     width = int(np.ceil((maxx - minx) / resolution))
    #     height = int(np.ceil((maxy - miny) / resolution))
    #     transform = rasterio.transform.from_origin(minx, maxy, resolution, resolution)
    # 
    #     raster = features.rasterize(
    #         ((mapping(geom), 1) for geom in (buildings_union.geoms if hasattr(buildings_union, 'geoms') else [buildings_union])),
    #         out_shape=(height, width),
    #         transform=transform,
    #         fill=0,
    #         dtype=np.uint8
    #     )
    # 
    #     return raster, transform
    # 
    # # -----------------------------
    # # Step 2: Compute IMI_unadj
    # # -----------------------------
    # def compute_imi_unadj(raster, resolution=5):
    #     """
    #     Compute unadjusted Index of Moment of Inertia for a raster.
    # 
    #     Parameters:
    #         raster (np.array): 2D raster array with 1 = building, 0 = empty
    #         resolution (float): raster cell size
    # 
    #     Returns:
    #         float: IMI_unadj value
    #     """
    #     rows, cols = np.where(raster == 1)
    #     if len(rows) == 0:
    #         return 0
    # 
    #     x_coords = cols * resolution + resolution / 2
    #     y_coords = rows * resolution + resolution / 2
    #     cx = np.mean(x_coords)
    #     cy = np.mean(y_coords)
    # 
    #     d_squared = (x_coords - cx)**2 + (y_coords - cy)**2
    #     s2 = resolution**2
    #     IMI_unadj = (s2 / (2 * np.pi)) * np.sum(s2 / 6 + d_squared)
    #     return IMI_unadj
    # 
    # # -----------------------------
    # # Step 3: Compute normalized compactness index for one subdivision
    # # -----------------------------
    # def ci_intra_for_subdivision(buildings, subdivision, resolution=1):
    #     """
    #     Compute normalized intra-subdivision compactness index (ci_intra).
    # 
    #     Handles Polygon and MultiPolygon subdivisions.
    #     """
    #     buildings_clipped = [geom.intersection(subdivision) for geom in buildings]
    #     buildings_clipped = [geom for geom in buildings_clipped if not geom.is_empty]
    # 
    #     if len(buildings_clipped) == 0:
    #         return None
    # 
    #     # Rasterize building footprints
    #     raster, _ = rasterize_buildings(buildings_clipped, subdivision, resolution)
    #     imi_value = compute_imi_unadj(raster, resolution)
    # 
    #     # Maximum possible inertia approximation
    #     sub_centroid = subdivision.centroid
    # 
    #     # Handle Polygon and MultiPolygon
    #     if subdivision.geom_type == 'Polygon':
    #         vertices = list(subdivision.exterior.coords)
    #     elif subdivision.geom_type == 'MultiPolygon':
    #         vertices = []
    #         for poly in subdivision.geoms:
    #             vertices.extend(list(poly.exterior.coords))
    #     else:
    #         raise ValueError(f"Unsupported geometry type: {subdivision.geom_type}")
    # 
    #     # Max distance from centroid to any vertex
    #     max_distance = max([sub_centroid.distance(Point(coord)) for coord in vertices])
    # 
    #     n_cells = np.sum(raster == 1)
    #     I_max = n_cells * (resolution**2 + max_distance**2)
    # 
    #     ci = imi_value / I_max if I_max > 0 else None
    #     return ci
    # 
    # # -----------------------------
    # # Step 4: Apply to all subdivisions with progress bar
    # # -----------------------------
    # def calculate_ci_intra_for_all(ABT, building_footprint_dataset, resolution=5):
    #     """
    #     Compute ci_intra for all subdivisions and add as a new column in ABT.
    # 
    #     Parameters:
    #         ABT (GeoDataFrame): Subdivisions with 'subd_id'
    #         building_footprint_dataset (GeoDataFrame): Building footprints
    #         resolution (float): raster cell size
    # 
    #     Returns:
    #         GeoDataFrame: ABT with 'ci_intra' column
    #     """
    #     ci_values = []
    # 
    #     # tqdm progress bar
    #     for idx, sub in tqdm(ABT.iterrows(), total=len(ABT), desc="Calculating ci_intra"):
    #         sub_poly = sub.geometry
    #         buildings_in_sub = building_footprint_dataset[building_footprint_dataset.geometry.intersects(sub_poly)]
    #         buildings_list = [geom for geom in buildings_in_sub.geometry if not geom.is_empty]
    # 
    #         ci = ci_intra_for_subdivision(buildings_list, sub_poly, resolution)
    #         ci_values.append(ci)
    # 
    #     ABT['ci_intra'] = ci_values
    #     return ABT
    # 
    # ABT = calculate_ci_intra_for_all(ABT, building_footprint_dataset, resolution=5) 
    
else:
    print("✅ Cohesion Index (Intra) already exist).")